In [1]:
from IPython.display import HTML
html1 = """
    <img src="https://images.theconversation.com/files/45159/original/rptgtpxd-1396254731.jpg" style="margin-left:auto; margin-right:auto"/>
"""
HTML(html1)

# A quick analysis of my reading habits

* What do I read the most?
* What format do I prefer?
* Are there any Genre-Sub-Genre pairs I tend to gravitate towards?
* Who is my most read author?
* How long does it take me to read books?

In [2]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

In [3]:
# define the scope
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

# add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name('book_creds.json', scope)

# authorize the clientsheet 
client = gspread.authorize(creds)

# get the instance of the Spreadsheet
sheetname = 'Book_List'
sheet = client.open(sheetname)

# get the first sheet of the Spreadsheet
sheet_instance = sheet.get_worksheet(0)

In [4]:
expected_headers = sheet_instance.row_values(1)
data = sheet_instance.get_values()
df = pd.DataFrame(data[1:], columns = expected_headers).drop(axis=1, columns=['Physical', 'eBook'])
df.head()

,Title,Author,Publisher,Genre,Sub-Genre,Year,Pages,Start Date,Finish Date,Duration,Rating
0,The War of Art,Steven Pressfield,Black Irish Entertrainment,Nonfiction,Self Help,2012,190,,,,3.0
1,The Sound of the Mountain,Yasunari Kawabata,Vintage,Fiction,Cultural,1996,235,04-21-2021,05-17-2021,27,4.5
2,The Four Agreements,Don Miguel Ruiz,Amber-Allen Publishing,Nonfiction,Self Help,1997,140,12-16-2020,12-16-2020,1,5.0
3,The Art of Learning,Josh Waitzkin,Free Press,Nonfiction,Self Help,2007,288,03-30-2021,04-04-2021,6,5.0
4,Almond: A Novel,Won Pyung Sohn,HarperVia,Fiction,Young Adult,2017,154,08-29-2021,08-30-2021,2,5.0


In [5]:
df[['Year', 'Pages']] = df[['Year', 'Pages']].apply(pd.to_numeric, downcast='integer')
df['Duration'] = df['Duration'].apply(pd.to_numeric, downcast='integer')
df['Rating'] = df['Rating'].apply(pd.to_numeric, downcast='float')
df['Start Date']= pd.to_datetime(df['Start Date'])
df['Finish Date']= pd.to_datetime(df['Finish Date'])
df = df[df['Rating'].notnull()] #removes any blank rows or books I never finished/rated

df.dtypes

Title                  object
Author                 object
Publisher              object
Genre                  object
Sub-Genre              object
Year                  float64
Pages                 float64
Start Date     datetime64[ns]
Finish Date    datetime64[ns]
Duration              float64
Rating                float32
dtype: object

In [6]:
df['Sub-Genre'].value_counts().head()

Sub-Genre
Historical Fiction    11
Nonfiction             8
Classics               8
Self Help              6
Cultural               5
Name: count, dtype: int64

In [7]:
df[df.isna().any(axis=1)]

,Title,Author,Publisher,Genre,Sub-Genre,Year,Pages,Start Date,Finish Date,Duration,Rating
0,The War of Art,Steven Pressfield,Black Irish Entertrainment,Nonfiction,Self Help,2012.0,190.0,NaT,NaT,NaN,3.0
7,How to Win Friends & Influence People,Dale Carnegie,Simon & Schuster,Nonfiction,Self Help,2011.0,276.0,NaT,NaT,NaN,5.5
14,Meditations,Marcus Aurelius,Random House Publishing Group,Philosophy,Nonfiction,2003.0,194.0,NaT,NaT,NaN,6.5
18,Selected Writings & Discourses,Epictetus,Penguin Classics,Philosophy,Nonfiction,2008.0,276.0,NaT,NaT,NaN,7.0
19,Letters From A Stoic,Seneca,Penguin Classics,Philosophy,Nonfiction,2004.0,254.0,NaT,NaT,NaN,7.0
22,Emotional Agility,Susan David,Avery Publishing Group,Psychology,Self Help,2016.0,288.0,NaT,NaT,NaN,7.5
30,Norwegian Wood,Haruki Murakami,Knopf Doubleday Publishing Group,Fiction,Cultural,2000.0,298.0,NaT,NaT,NaN,8.5
